In [1]:
import pandas as pd
import os
from shapely.geometry import Point, Polygon
from shapely import geometry
import ast

#### Reading In Relevant Files

In [2]:
directory = '../../data/processed'
void = ['clean_areas.csv', 'clean_disadvantaged_areas.csv', 'clean_police_districts.csv', 'clean_public_healthindicator.csv', 'clean_train_ridership.csv', 'clean_bike_trips.csv']

def read_in(directory, void):
    data = {}
    for filename in os.listdir(directory):
        if filename not in void and filename.endswith('.csv'):
            file_path = os.path.join(directory, filename)
            data[filename[:-4]] = pd.read_csv(file_path)
            print(f'{filename[:-4]} successfully read in')
    return data

In [3]:
data = read_in(directory, void)

clean_311 successfully read in
clean_bike_stations successfully read in
clean_bus_stops successfully read in
clean_crime successfully read in
clean_police_stations successfully read in
clean_train_stations successfully read in
clean_vacant_buildings successfully read in


In [4]:
clean_police_districts = pd.read_csv('../../data/processed/clean_police_districts.csv')
clean_public_healthindicator = pd.read_csv('../../data/processed/clean_public_healthindicator.csv')
clean_areas = pd.read_csv('../../data/processed/clean_areas.csv')
clean_disadvantaged_areas = pd.read_csv('../../data/processed/clean_disadvantaged_areas.csv')

#### Data Cleaning of Geom Types

In [5]:
def parse_polygon1(polygon_string):
    points = polygon_string.strip('POLYGON ((').strip('))').split(', ')
    points = [tuple(map(float, point.split())) for point in points]
    return Polygon(points)

In [6]:
def parse_polygon2(polygon_string):
    points = ast.literal_eval(polygon_string)
    return Polygon(points)

In [7]:
def swap_coordinates(polygon):
    if polygon.is_empty:
        return polygon
    swapped_coords = [(y, x) for x, y in polygon.exterior.coords]
    return Polygon(swapped_coords)

In [8]:
clean_police_districts['geom'] = clean_police_districts['geom'].apply(parse_polygon1)
clean_police_districts.head()

,district,geom
0,17,POLYGON ((41.997365655369435 -87.7106708939135...
1,20,POLYGON ((41.990916338539776 -87.6602942357235...
2,31,POLYGON ((41.98384222028714 -87.82817787271652...
3,31,POLYGON ((41.97535481526603 -87.83365455160555...
4,19,POLYGON ((41.969727093814825 -87.6449179856867...


In [9]:
clean_areas['poly'] = clean_areas['poly'].apply(parse_polygon2)
clean_areas.head()

,id,poly
0,35,POLYGON ((41.84469250265398 -87.60914087617894...
1,36,POLYGON ((41.81692934626684 -87.59215283879394...
2,37,POLYGON ((41.80189303368919 -87.62879823733725...
3,38,"POLYGON ((41.81681377057218 -87.6067081256125,..."
4,39,POLYGON ((41.81692934626684 -87.59215283879394...


In [10]:
clean_disadvantaged_areas['poly'] = clean_disadvantaged_areas['poly'].apply(parse_polygon2)
clean_disadvantaged_areas['poly'] = clean_disadvantaged_areas['poly'].apply(swap_coordinates)
clean_disadvantaged_areas.head()

,poly
0,POLYGON ((41.707966000031426 -87.5736629999675...
1,POLYGON ((41.68632499999862 -87.60151899995586...
2,POLYGON ((41.69284299996604 -87.60036199996543...
3,POLYGON ((41.69942700001048 -87.62032800000847...
4,POLYGON ((41.75771799998948 -87.66036699995497...


In [11]:
clean_areas['centroid'] = clean_areas['poly'].apply(lambda poly : poly.centroid)
clean_areas

,id,poly,centroid
0,35,POLYGON ((41.84469250265398 -87.60914087617894...,POINT (41.835118342623176 -87.61867772050564)
1,36,POLYGON ((41.81692934626684 -87.59215283879394...,POINT (41.823750345730396 -87.60321641296161)
2,37,POLYGON ((41.80189303368919 -87.62879823733725...,POINT (41.8090854843581 -87.63242456957649)
3,38,"POLYGON ((41.81681377057218 -87.6067081256125,...",POINT (41.81294935894828 -87.6178596907689)
4,39,POLYGON ((41.81692934626684 -87.59215283879394...,POINT (41.808916370166855 -87.59618357877342)
...,...,...,...
72,74,POLYGON ((41.70714491233857 -87.69645961375822...,POINT (41.69487944254784 -87.7131918580596)
73,75,POLYGON ((41.685082119670845 -87.6421520465139...,POINT (41.689729585922855 -87.66905398718522)
74,76,POLYGON ((41.986396111591276 -87.8365808787436...,POINT (41.977778053264366 -87.89466582726284)
75,77,POLYGON ((41.99816614970252 -87.65455590025104...,POINT (41.98671222550283 -87.6634166067842)


In [12]:
clean_public_healthindicator = pd.merge(left=clean_public_healthindicator, right=clean_areas[['id','centroid']], on='id', how='left')
clean_public_healthindicator.drop('id', axis=1, inplace=True)
clean_public_healthindicator.head()

,unemployment,per_capita_income,no_hs_dip,gov_depend,crowded_housing,below_pov,centroid
0,0.075,23714,0.181,0.288,0.079,0.227,POINT (42.009623377988696 -87.6701672940393)
1,0.079,21375,0.196,0.383,0.070,0.151,POINT (42.001572146349325 -87.69501265887241)
2,0.077,32355,0.136,0.222,0.046,0.227,POINT (41.96581207771316 -87.65587931915148)
3,0.068,35503,0.125,0.256,0.031,0.095,POINT (41.97517153048752 -87.68751544295701)
4,0.045,51615,0.054,0.255,0.002,0.071,POINT (41.94779237360123 -87.68383517981742)


In [13]:
clean_disadvantaged_areas['centroid'] = clean_disadvantaged_areas['poly'].apply(lambda poly : poly.centroid)
clean_disadvantaged_areas

,poly,centroid
0,POLYGON ((41.707966000031426 -87.5736629999675...,POINT (41.683509683928065 -87.57231914155308)
1,POLYGON ((41.68632499999862 -87.60151899995586...,POINT (41.68931505835441 -87.60536971084375)
2,POLYGON ((41.69284299996604 -87.60036199996543...,POINT (41.70085594037496 -87.60030206564569)
3,POLYGON ((41.69942700001048 -87.62032800000847...,POINT (41.696058183380245 -87.6162676498575)
4,POLYGON ((41.75771799998948 -87.66036699995497...,POINT (41.754048514350146 -87.66110569971308)
...,...,...
275,POLYGON ((41.998169000012545 -87.6624109999806...,POINT (42.00032005328326 -87.66312691441664)
276,POLYGON ((42.00497899998903 -87.66572099996036...,POINT (42.00451710898514 -87.66332746402558)
277,POLYGON ((41.998316999982876 -87.6550000000143...,POINT (41.99727725336176 -87.65564251215129)
278,POLYGON ((41.99662699998114 -87.65029099995624...,POINT (41.99538042311063 -87.65469543980272)


#### Assigning Districts to Each Dataset

In [14]:
def determine_within(df):

    statuses = []
    districts = []
    cent = True if 'centroid' in df.columns else False

    for i in range(len(df)):
        point = df.centroid.loc[i] if cent else geometry.Point(df.lat.loc[i], df.long.loc[i])
        status = 0

        for index, row in clean_police_districts.iterrows():
            district = row['district']
            geom = row['geom']
            if (geom.contains(point)): 
                status = 1
                break
        statuses.append(status)
        districts.append(district)

    df['status'] = statuses
    df['district'] = districts
    df = df[df['status'] == 1].drop('status', axis=1)

    if cent: df.drop('centroid', axis=1, inplace=True)

    return df

In [15]:
def determine_districts(data):
    for df_name, df in data.items():
        data[df_name] = determine_within(df)
        print(f'{df_name} successfully completed')
    return data

In [16]:
clean_disadvantaged_areas = determine_within(clean_disadvantaged_areas)
clean_public_healthindicator = determine_within(clean_public_healthindicator)

In [17]:
data = determine_districts(data)

clean_311 successfully completed
clean_bike_stations successfully completed
clean_bus_stops successfully completed
clean_crime successfully completed
clean_police_stations successfully completed
clean_train_stations successfully completed
clean_vacant_buildings successfully completed


#### Data Cleaning to Finalize Datasets

In [18]:
clean_bike_trips = pd.read_csv('../../data/processed/clean_bike_trips.csv')
clean_train_ridership = pd.read_csv('../../data/processed/clean_train_ridership.csv')

In [19]:
clean_bike_trips = clean_bike_trips.merge(right=data['clean_bike_stations'][['id', 'district']], how='left', left_on='station_id', right_on='id').dropna(subset=['district'])
clean_train_ridership = clean_train_ridership.merge(right=data['clean_train_stations'][['station_name', 'district']], how='left', on='station_name').dropna(subset=['district'])

In [20]:
clean_police_districts = clean_police_districts.merge(right=clean_disadvantaged_areas.groupby('district').agg('count'), how='left', on='district').fillna(0).rename(columns={'poly':'disadvantaged_score'})
clean_police_districts

,district,geom,disadvantaged_score
0,17,POLYGON ((41.997365655369435 -87.7106708939135...,0.0
1,20,POLYGON ((41.990916338539776 -87.6602942357235...,4.0
2,31,POLYGON ((41.98384222028714 -87.82817787271652...,0.0
3,31,POLYGON ((41.97535481526603 -87.83365455160555...,0.0
4,19,POLYGON ((41.969727093814825 -87.6449179856867...,2.0
5,25,POLYGON ((41.93921621331353 -87.71739917372257...,7.0
6,14,POLYGON ((41.93942656894302 -87.69256666010364...,3.0
7,31,POLYGON ((41.69898580247497 -87.69123096883077...,0.0
8,22,"POLYGON ((41.7361768878879 -87.63631615604635,...",9.0
9,5,POLYGON ((41.72230545260696 -87.58775992207175...,16.0


#### Save Datasets with District Data

In [21]:
clean_bike_trips.to_csv('../../data/processed/clean_bike_trips.csv', index=False)
clean_train_ridership.to_csv('../../data/processed/clean_train_ridership.csv', index=False)
clean_police_districts.to_csv('../../data/processed/clean_police_districts.csv', index=False)

In [22]:
def save_data():
    for df_name, df in data.items():
        df.to_csv(f'../../data/processed/{df_name}.csv', index=False)

In [23]:
save_data()

- Gather district geom data
- Assign district to all locational data
- Create a master dataset with all features
    - based on crime dataset
    - attach aggregated data from each other dataset based on time and district of each crime
    - attach lag and leads of crime, bike, cta, and 311 data 